# PyTorch练习2

In [1]:
#请输出你的姓名
print('赵旭阳')

赵旭阳


In [2]:
import torch
print(torch.__version__)

1.9.0


## 1.糖尿病预测

In [97]:
from sklearn.datasets import load_diabetes
diabetes_dataset=load_diabetes()


data=diabetes_dataset['data']

targets=diabetes_dataset['target']
print(data.shape)
print(targets.shape)

(442, 10)
(442,)


In [111]:

import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split 

x = torch.tensor(data, dtype = torch.float32)
y = torch.tensor(targets, dtype = torch.float32)


x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

logReg = nn.Linear(10, 1)

def train(x_train, y_train,epochs=100,lr=0.1, momentum=0.9):
    loss_fn = nn.MSELoss(reduction='mean')
    
    logReg.train()
    optimizer = optim.SGD(params=logReg.parameters(), lr=lr, momentum=momentum)
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = logReg(x_train)
        loss = loss_fn(y_pred, y_train.view(y_train.shape[0],1))
        loss.backward()
        optimizer.step()
        

        w = logReg.get_parameter('weight').data
        b = logReg.get_parameter('bias').data
        

        dist = (y_pred - y_train.view(y_train.shape[0],1)).mean()
        print('Epoch = {0}, Loss = {1}, w = {2}, b = {3}, mean_dist = {4:.2f}'.format(epoch, loss.detach().numpy(), w.numpy().squeeze(), b.numpy(), dist.item()))#
    return w,b

def test(x_test, y_test, w, b):
    y_pred = logReg(x_test)
    dist = (y_pred - y_test.view(y_test.shape[0],1)).mean()
    print('test_mean_dist:{:.2f}'.format(dist.item()))
    
w,b = train(x_train, y_train, epochs=100, lr=0.01, momentum=0.9)
test(x_test,y_test,w,b)


Epoch = 0, Loss = 29466.5234375, w = [-0.04397763 -0.14235923  0.02343526 -0.10702411 -0.22645608 -0.09062316
 -0.07780288  0.0916943  -0.08181348 -0.22062364], b = [2.921936], mean_dist = -152.24
Epoch = 1, Loss = 28547.806640625, w = [-0.02125069 -0.12845385  0.1058058  -0.04159489 -0.19316575 -0.06620592
 -0.13652854  0.1600335   0.00256831 -0.16466773], b = [8.64625], mean_dist = -149.20
Epoch = 2, Loss = 26870.787109375, w = [ 0.01125436 -0.10877232  0.22346379  0.05173074 -0.14566757 -0.03132378
 -0.22035024  0.25744602  0.12302777 -0.08475972], b = [16.667603], mean_dist = -143.47
Epoch = 3, Loss = 24631.0234375, w = [ 0.05265941 -0.08406246  0.37307224  0.17016609 -0.08536501  0.01303856
 -0.32682568  0.3809562   0.27607358  0.01681358], b = [26.595873], mean_dist = -135.45
Epoch = 4, Loss = 22036.916015625, w = [ 0.10219756 -0.05503796  0.5516729   0.31120336 -0.0135176   0.06600913
 -0.4537723   0.52786905  0.4585905   0.1380193 ], b = [38.04181], mean_dist = -125.52
Epoch = 

test_mean_dist:-0.27


## 2. MINST手写体识别

In [124]:
from torchvision import datasets,transforms

data_path = '../data/'
mnist_train = datasets.MNIST(data_path,download=True,train = True,transform = transforms.ToTensor())
mnist_test =  datasets.MNIST(data_path,download=True,train = False,transform = transforms.ToTensor())


print(len(list(mnist_train)))
print(len(list(mnist_test)))

60000
10000


In [130]:

mnist_mlp = nn.Sequential(nn.Linear(784,64),nn.Sigmoid(),nn.Linear(64,10))

optimizer = optim.SGD(mnist_mlp.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = 64, shuffle = True)

epochs = 10

mnist_mlp.train()
for epoch in range(epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        logits = mnist_mlp(imgs.view(batch_size, -1))
        loss = loss_fn(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print('train success')
        
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = 100, shuffle = False)

mnist_mlp.eval()
correct = 0
total = 0        

with torch.no_grad():
    for imgs, labels in test_loader:
        batch_size = imgs.shape[0]
        logits = mnist_mlp(imgs.view(batch_size, -1))
        _,predicted = torch.max(logits.data,1)
        total += batch_size
        correct += (predicted == labels).sum().item()
        
acc = correct / total
print('准确度：\n {}'.format(acc))

train success
准确度：
 0.8922
